# Calculate zonal statistics

This is the fifth notebook in a series of six notebooks designed to run sequentially. \
Run this notebook to calculate zonal statistics (mean) for each model set-model combination and for each multimodel weighted ensemble. \
This notebook uses REST services for the zone features. \
Insert your desired feature service URL below under 'Your feature service'. \
Example zones are included for counties, watershed boundaries, and tribal areas for CONUS.

In [ ]:
import arcpy
import os
import glob
import shutil
import numpy as np
from arcgis.gis import GIS

In [ ]:
# Fill in your username and password for the GIS connection
gis = GIS('https://nationalclimate.maps.arcgis.com/', username='*****', password='*****')

# Define functions

In [3]:
def remove_folder(name):

    if arcpy.Exists(name):
    
        shutil.rmtree(name)

In [4]:
def remove_file(name):    

    files = glob.glob(os.path.join(name + '*'))

    for file in files:

        if file in glob.glob(os.path.join(name + '*.lock')):

            continue

        else:
            
            os.remove(file)

# Set paths and variables

In [ ]:
# Path to your base directory
base = r'C:\<your-base-path>\CRIS'
out_path = os.path.join(base, 'zonalstats_REST')

# Set the workspace and overwrite output
arcpy.env.workspace = out_path
arcpy.env.overwriteOutput = True

In [6]:
# pick 'STAR', 'LOCA2', 'STAR_ensemble', 'LOCA2_ensemble', or 'LOCA2STAR_ensemble'
model_set_start = 'LOCA2STAR_ensemble'

# pick model (only for STAR and LOCA2)
model = 'ACCESS-CM2'

# Pick one or more variables
variables = ['cdd']

## Select SSP Scenario

In [8]:
ssp = '245'
# ssp = '585'

## Select time period

In [9]:
# Pick start and end year of the period you want to process.
# If you only want to process 1 year, put your desired year for both the start and end.

year_start = 2040
year_end = 2040

time_start = f'{year_start}-12-31T00:00:00'
time_end = f'{year_end}-12-31T00:00:00'

## Feature Service. Only run the cell with your preferred feature layer.

##### USA 119th Congressional Districts

In [ ]:
feat_srvc = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_119th_Congressional_Districts/FeatureServer/0'

feat_name = feat_srvc.split('/')[-3]

# Use a uniquely defining field to pick which USA Congressional District(s) to calculate zonal statistics over.
# In this case, use district IDs (Field Name: DISTRICTID).
uniqueID = 'DISTRICTID'

# Put each district ID between apostrophes and separate by commas.
# Set ids to 'all' if using all disctrict IDs in the feature layer
ids = '0634', '0635', '0636', '0101', '0102'
# ids = 'all'

##### USA ZIP Code Boundaries 2023

In [10]:
feat_srvc = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Boundaries_2023/FeatureServer/3'

feat_name = feat_srvc.split('/')[-3]

# Use a uniquely defining field to pick which ZIP code(s) to calculate zonal statistics over.
# In this case, use the base names, i.e. ZIP codes (Field Name: BASENAME).
# uniqueID = 'BASENAME'
uniqueID = 'ZIP_CODE'

# Put each ZIP code between apostrophes and separate by commas.
# Set ids to 'all' if using all ZIP codes in the feature layer
ids = '90025', '90026', '90027', '90028', '90029', '90030', '90031'
# ids = 'all'

##### Watershed Boundary Dataset HUC 6s

In [ ]:
feat_srvc = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/Watershed_Boundary_Dataset_HUC_6s/FeatureServer/0'

feat_name = feat_srvc.split('/')[-3]

# Use a uniquely defining field to pick which watershed boundaries to calculate zonal statistics over.
# In this case, use the watershed boundary name (Field Name: NAME).
uniqueID = 'NAME'

# Put each watershed boundary name between apostrophes and separate by commas.
# Set ids to 'all' if using all watershed boundaries in the feature layer
ids = 'Santa Ana', 'Wabash', 'Santa Cruz', 'Southern Mojave'
# ids = 'all'

##### Your feature service

In [ ]:
feat_srvc = '<your-feature-service-url>' # don't forget to add your layer number at the end of the url, e.g. '/0'

feat_name = feat_srvc.split('/')[-3]

# Use a uniquely defining field to pick which feature(s) to calculate zonal statistics over.
uniqueID = ''

# Put each ID between apostrophes and separate by commas.
# Set ids to 'all' if using all features in the feature layer
ids = ''
# ids = 'all'

## Define (subset of) feature layer

In [11]:
if ids == 'all':
    
    feat_layer = feat_srvc

else:
    
    if np.size(ids) == 1:

        where_clause_id = f"{uniqueID} = '{ids}'"
        feat_layer = arcpy.management.MakeFeatureLayer(feat_srvc, 'feat_layer', where_clause_id)

    else:

        where_clause_id = f"{uniqueID} IN {ids}"
        feat_layer = arcpy.management.MakeFeatureLayer(feat_srvc, 'feat_layer', where_clause_id)

## Calculate zonal statistics

In [ ]:
for var in variables:
    
    print('Variable being processed:', var)
    print('Setting variable and path names')

    # Create output geodatabase names
    gdb_name_temp = feat_name + '_temp.gdb'
    gdb_name = feat_name + '.gdb'

    # Create output geodatabase paths
    out_gdb_path_temp = os.path.join(out_path, gdb_name_temp)
    out_gdb_path = os.path.join(out_path, gdb_name)
    
    # Create output geodatabases if they do not exist
    if not arcpy.Exists(out_gdb_path_temp):
        arcpy.management.CreateFileGDB(out_path, gdb_name_temp)
    
    if not arcpy.Exists(out_gdb_path):
        arcpy.management.CreateFileGDB(out_path, gdb_name)
    
    # Change variable name to a format that is compatible with file names
    var_format = var.replace('-','_').replace(' ', '_').replace(',','').replace('(','')

    # Set output file and geodatabase table names
    crf_out = os.path.join(out_path, var_format + '.crf')
    out_gdb_table = os.path.join(out_gdb_path_temp, var_format + f'_{year_start}_{year_end}')    
    
    # Create path names for STAR and LOCA2 model sets
    if model_set_start == 'STAR' or model_set_start =='LOCA2':

        # Create path name based on model_set_start and variable
        netcdf_dir_list = os.path.join(base, 'data', 'resample_mask', model_set_start, var)

        # Create netcdf name based on model_set_start, model, variable, and ssp
        netcdf_name = os.path.join(netcdf_dir_list, f'{model_set_start}_{model}_{var_format}_ssp{ssp}_1950_2100.nc')

    # Create path names for STAR_ensemble, LOCA2_ensemble, and LOCA2STAR_ensemble model sets
    elif model_set_start == 'STAR_ensemble' or model_set_start == 'LOCA2_ensemble' or model_set_start == 'LOCA2STAR_ensemble':

        # Remove '_ensemble' from model_set_start
        model_set_start_format = model_set_start.strip('_ensemble')

        # Create path name based on model_set_start
        netcdf_dir_list = os.path.join(base, 'data', 'ensemble', model_set_start_format)

        # Create netcdf name based on model_set_start, variable, and ssp
        netcdf_name = os.path.join(netcdf_dir_list, f'{model_set_start_format}_{var_format}_ssp{ssp}_1950_2100.nc')

    # Strip .nc from the netcdf_name and add .crf
    crf_name = f'{model_set_start_format}_{var_format}_ssp{ssp}_1950_2100.crf'

    # Copy the netcdf file to a crf file with the same name
    print('Processing: CopyRaster')
    arcpy.management.CopyRaster(netcdf_name, os.path.join(out_path, crf_name), process_as_multidimensional='ALL_SLICES', build_multidimensional_transpose='TRANSPOSE')

    # Use the copied crf file as the input for SubsetMultidimensionalRaster
    img_srvc = os.path.join(out_path, crf_name)

    # Subset variable and time
    print('Processing: SubsetMultidimensionalRaster 1')
    arcpy.md.SubsetMultidimensionalRaster(img_srvc,
                                          var_format,
                                          variables=var,
                                          dimension_def = 'BY_RANGES',
                                          dimension_ranges = 'StdTime ' + time_start + ' ' + time_end
                                          )

    # Calculate Zonal Stastics
    print('Processing: ZonalStatisticsAsTable 1')
    arcpy.ia.ZonalStatisticsAsTable(feat_layer, uniqueID, crf_out, out_gdb_table, ignore_nodata='DATA', statistics_type='MEAN', process_as_multidimensional='ALL_SLICES')

    # Some features can be very small and might not be captured by the first Zonal Statistics As Table. These need to be converted to points and then re-processed.
    # Check if number of features in the output gdb is equal to the number of features in the input feature layer
    # If not, the below code (after "elif") will add the missing features to the output gdb
    if int(arcpy.management.GetCount(feat_layer).getOutput(0)) == int(arcpy.management.GetCount(out_gdb_table).getOutput(0)):

        # Don't sort if there is only one feature in out_gdb_table
        if int(arcpy.management.GetCount(out_gdb_table).getOutput(0)) != 1:

            # Sort the zonal statistics table
            print('Processing: Sort')
            output_all_sorted = os.path.join(out_gdb_path, var_format + f'_SSP{ssp}' + f'_{year_start}_{year_end}')
            arcpy.management.Sort(out_gdb_table, output_all_sorted, [[uniqueID, 'ASCENDING']]);

    elif int(arcpy.management.GetCount(feat_layer).getOutput(0)) != int(arcpy.management.GetCount(out_gdb_table).getOutput(0)):

        # Raster to Point using your input value raster (this raster has the desired output spatial resolution)
        print('Processing: RasterToPoint')
        arcpy.conversion.RasterToPoint(crf_out, os.path.join(out_path, 'RtoP.shp'), 'Value');

        # Spatial Join single part features from #2 (Join Features) to the grid points from #1 (Target Features).
        # Join One-to-one & uncheck Keep all target features.
        # Unchecking the Keep all target features results in some centroid points being removed from the output if they do not intersect with a polygon.
        print('Processing: SpatialJoin')
        arcpy.analysis.SpatialJoin(os.path.join(out_path, 'RtoP'), feat_layer, os.path.join(out_path, 'features_and_RtoP'), 'JOIN_ONE_TO_ONE', 'KEEP_COMMON');

        # Select by Location single part features from #2 that do NOT intersect results from #3.
        print('Processing: SelectLayerByLocation')
        feat_layer_select_location = arcpy.management.SelectLayerByLocation(feat_layer, 'INTERSECT', os.path.join(out_path, 'features_and_RtoP'), invert_spatial_relationship='INVERT');

        # Feature to Point the results from #4. Check Inside.
        print('Processing: FeatureToPoint')
        arcpy.management.FeatureToPoint(feat_layer_select_location, os.path.join(out_path, 'feat_layer_FtoP'), point_location='INSIDE');

        # Do zonalstats for extra points not captured by the first Zonal Statistics As Table
        print('Processing: ZonalStatisticsAsTable 2')
        temp_table = os.path.join(out_gdb_path_temp, var_format + f'_SSP{ssp}' + f'_{year_start}_{year_end}_added_points_pre_merge')
        arcpy.ia.ZonalStatisticsAsTable(os.path.join(out_path, 'feat_layer_FtoP'), uniqueID, crf_out, temp_table, ignore_nodata='DATA', statistics_type='MEAN', process_as_multidimensional='ALL_SLICES');

        # merge two zonalstatistics (features and points)
        print('Processing: Merge')
        input_most = out_gdb_table
        input_few = temp_table
        output_all = os.path.join(out_gdb_path_temp, var_format + f'_SSP{ssp}' + f'_{year_start}_{year_end}_merge')
        arcpy.management.Merge([input_most, input_few], output_all);

        # sort the merged zonalstatistics
        print('Processing: Sort')
        output_all_sorted = os.path.join(out_gdb_path, var_format + f'_SSP{ssp}' + f'_{year_start}_{year_end}')
        arcpy.management.Sort(output_all, output_all_sorted, [[uniqueID, 'ASCENDING']]);

    # Remove all temporary folders and files
    print('Removing all temporary folders and files')
    remove_folder(crf_out)
    remove_file(os.path.join(out_path,'RtoP'))
    remove_file(os.path.join(out_path,'feat_layer_FtoP'))
    remove_file(os.path.join(out_path,'features_and_RtoP'))
    arcpy.management.Delete(out_gdb_path_temp);